In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [83]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
#train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
#test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
one_data = datasets.ImageFolder(data_dir+'/one', transform=train_transforms)
#creare una directory sotto content Cat_dog_data, un'altra one, creare un'altra sottodirectory cat e 
#inserire una foto jpeg

#trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
#testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
oneloader = torch.utils.data.DataLoader(one_data, batch_size=1)

#print(datasets.ImageFolder)
print(one_data[0])


(tensor([[[ 0.3823,  0.3823,  0.3994,  ..., -2.1179, -2.1179, -2.1179],
         [ 0.3994,  0.3994,  0.3994,  ..., -2.1179, -2.1179, -2.1179],
         [ 0.3994,  0.3994,  0.3823,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-1.6555, -1.6384, -1.6042,  ...,  0.3994,  0.3652,  0.3652],
         [-1.5870, -1.5870, -1.5528,  ...,  0.4166,  0.3994,  0.3994],
         [-1.5357, -1.5357, -1.5014,  ...,  0.4337,  0.4166,  0.4166]],

        [[ 0.5203,  0.5203,  0.5378,  ..., -2.0357, -2.0357, -2.0357],
         [ 0.5378,  0.5378,  0.5378,  ..., -2.0357, -2.0357, -2.0357],
         [ 0.5378,  0.5378,  0.5203,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-1.5980, -1.5805, -1.5455,  ...,  0.6078,  0.5728,  0.5728],
         [-1.5280, -1.5280, -1.4930,  ...,  0.6254,  0.6078,  0.6078],
         [-1.4930, -1.4930, -1.4405,  ...,  0.6429,  0.6254,  0.6254]],

        [[ 0.5311,  0.5311,  0.5485,  ..., -1.8044, -1.8044, -1.8044],
         [ 0.5485,  0.5485,  0.5485,  ..., -

In [62]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [64]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in oneloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        print(logps)

        running_loss += loss.item()
        
        #if steps % print_every == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
                for inputs, labels in oneloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
        print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(oneloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(oneloader):.3f}")
        running_loss = 0
        model.train()
#print(inputs.shape)
#print(inputs)
#type(inputs)
#print(logps)
#print(labels[1])
#print(test_loss)
#print(equals)
#print(top_p)
#print(top_class)
#print(one_data)
#print(oneloader)

tensor([[-0.7892, -0.6055]], grad_fn=<LogSoftmaxBackward>)
Epoch 1/1.. Train loss: 0.121.. Test loss: 0.013.. Test accuracy: 1.000
